# Laboratory Work №6
## Manipulator Robot Control  
### ISU ID :


## Task  

1. Load the manipulator model from the toolbox according to the selected robot kinematics variant.  
2. Fill in all robot model parameters in accordance with Lab Work No. 1.  
3. Define the desired trajectory of the manipulator motion in accordance with Lab Work No. 2.  
4. Create a control system model for the robot that implements regulation based on impedance control.  
5. Tune the controller coefficients to minimize the end-effector position error.  
6. Plot the position, velocity, and torque for each robot link during motion along the trajectory.  
7. Add a payload and repeat steps 5–6.  
8. Prepare a report in `.ipynb` format with detailed comments. Draw conclusions based on the results of the work.  


## Execution  

### 0. Import the necessary libraries  

In [1]:
from math import pi
import numpy as np
import matplotlib.pyplot as plt
import roboticstoolbox as rtb
import spatialmath as sm

### 1. Import the IRB140 model (Puma doesn't work here)


In [1]:
robot = rtb.models.DH.IRB140()
robot

NameError: name 'rtb' is not defined

### 2. Fill in the robot model parameters  
>"You must adapt it for your model." 

Set the link masses 

In [3]:
robot.links[0].m = 20
robot.links[1].m = 17.4
robot.links[2].m = 4.8
robot.links[3].m = 0.82
robot.links[4].m = 0.34
robot.links[5].m = 0.09

Set the center of mass of each link  

In [4]:
robot.links[0].r = [0, 0, 0]
robot.links[1].r = [-0.3638, 0.006, 0.2275]
robot.links[2].r = [-0.0203, -0.0141, 0.07]
robot.links[3].r = [0, 0.019, 0]
robot.links[4].r = [0, 0, 0]
robot.links[5].r = [0, 0, 0.032]

Set the inertia tensor of each link. Format: [Lxx, Lyy, Lzz, Lxy, Lyz, Lxz]  

In [5]:
robot.links[0].I = [0, 0.35, 0, 0, 0, 0]
robot.links[1].I = [0.13, 0.524, 0.539, 0, 0, 0]
robot.links[2].I = [0.066, 0.086, 0.0125, 0, 0, 0]
robot.links[3].I = [0.0018, 0.0013, 0.0018, 0, 0, 0]
robot.links[4].I = [0.0003, 0.0004, 0.0003, 0, 0, 0]
robot.links[5].I = [0.00015, 0.00015, 0.00004, 0, 0, 0]

Set the drive inertia

In [6]:
robot.links[0].Jm = 0.0004
robot.links[1].Jm = 0.0004
robot.links[2].Jm = 0.0004
robot.links[3].Jm = 0.000033
robot.links[4].Jm = 0.000033
robot.links[5].Jm = 0.000033

Set the viscous friction coefficient of the drive  

In [7]:
robot.links[0].B = 0.0015
robot.links[1].B = 0.000817
robot.links[2].B = 0.00138
robot.links[3].B = 0.0000712
robot.links[4].B = 0.0000826
robot.links[5].B = 0.0000367

Set the Coulomb friction coefficient of the drive  

In [8]:
robot.links[0].Tc = [0.395, -0.435]
robot.links[1].Tc = [0.126, -0.071]
robot.links[2].Tc = [0.132, -0.105]
robot.links[3].Tc = [0.0112, -0.0169]
robot.links[4].Tc = [0.00926, -0.0145]
robot.links[5].Tc = [0.00396, -0.0105]

Set the gear ratio for each link  

In [9]:
robot.links[0].G = -62.6111
robot.links[1].G = 107.8150000000000
robot.links[2].G = -53.7063
robot.links[3].G = 76.0364
robot.links[4].G = 71.923
robot.links[5].G = 76.686

Set limits on the generalized coordinates for each link  

In [10]:
robot.links[0].qlim = [-2.792526803190927, 2.792526803190927]
robot.links[1].qlim = [-0.785398163397448, 3.926990816987241]
robot.links[2].qlim = [-3.926990816987241, 0.785398163397448]
robot.links[3].qlim = [-1.919862177193763, 2.967059728390360]
robot.links[4].qlim = [-1.745329251994330, 1.745329251994330]
robot.links[5].qlim = [-4.642575810304916, 4.642575810304916]

### 3. Define the desired trajectory  

Set the initial and final generalized coordinates  

In [11]:
q_start = [0, 0, 0, 0, 0, 0]
q_end = [pi/3, -pi/4, -pi/3, pi/4, -pi/3, pi/4]

Set the time array  

In [12]:
t_start, t_stop, t_shag = 0, 5, 0.05
time = np.arange(t_start,t_stop,t_shag)

Obtain the trajectory of the entire manipulator  

In [13]:
traj = rtb.mtraj(rtb.trapezoidal, q_start, q_end, time)
cq_traj = robot.fkine(traj.q)
cqd_traj = robot.fkine(traj.qd)

### 4. Synthesize the control loop  

Set coefficients for impedance control (three 6x6 matrices)  

In [14]:
Md = np.diag([1, 1, 1, 1, 1, 1], 0)  # everything here  
Kd = np.diag([2, 2, 2, 2, 2, 2], 0)  # is chosen arbitrarily,  
Kp = np.diag([3, 3, 3, 3, 3, 3], 0)  # needs to be tuned 

Initialize arrays for data storage  

In [15]:
torques = []
errors = []

define an impedance_controller

In [ ]:
def impedance_control(robot, t, q_act, qd_act):

    # Determine the array element (index) corresponding to the current time     
    mask = np.round(np.abs(time-t),3) <= t_shag
    list_idx = np.array(range(len(mask)))
    idx = list_idx[mask][0]

    # Extract position and velocity at the current time 
    q_des = traj.q[idx]
    qd_des = traj.qd[idx]
    
    # Solve the inverse kinematics for the current position  
    T_act = robot.fkine(q_act)
    T_des = robot.fkine(q_des)

    # Find the analytical Jacobians  
    J_act = np.round(robot.jacob0_analytical(q_act, representation='eul'),4)
    J_des = np.round(robot.jacob0_analytical(q_des, representation='eul'),4)
    Jd_act = np.round(robot.jacob0_dot(q_act, qd_act, representation='eul'),4)
    invJ_act = np.round(np.linalg.pinv(J_act),4)

    # Create vectors to calculate position errors 
    x_e = np.empty(6)
    x_e[:3] = T_act.A[:3,-1]
    x_e[3:] = sm.base.tr2eul(T_act.R)
    x_d = np.empty(6)
    x_d[:3] = T_des.A[:3,-1]
    x_d[3:] = sm.base.tr2eul(T_des.R)

    # Create vectors to calculate velocity errors  
    xd_e = J_act @ qd_act
    xd_d = J_des @ qd_des

    # Set the desired acceleration  
    xdd_des = np.array([0,0,0,0,0,0]) 

    # Solve it
    tau = 

    # Save values for plotting (You will need to figure out how to plot them correctly)  
    errors.append(x_d - x_e)
    torques.append(tau)

    return tau

 Solve the forward dynamics problem

In [17]:
tg = robot.fdyn(t_stop, q_start, impedance_control, solver="RK45", dt=t_shag, progress=True, solver_args={"atol":1e-3,"rtol":1e-2})
cq_tg = robot.fkine(tg.q)
cqd_tg = robot.fkine(tg.qd)

### 6. Plot graphs

Add plot the position graph  

Add plot velocity graph